# NLP for querying  (Open)Data

Il presente notebook è stato realizzato per partecipare ad una serata al PythonBiellaGroup. Ha pertanto contenuto didattico/espositivo.

Anche la ridondanza del codice e la scelta degli algoritmi da impiegare è propedeutica all'esposizione.

Come tutti sanno i dati possono distinguersi in 3 categorie:
1. **dati strutturati** (_es: file Excel_)
2. **dati non strutturati** (_es: testo_)
3. **dati semi-strutturati** (_es: pagina HTML_)

Estrarre i dati dai primi è molto semplice, non è la stessa cosa per i secondi...

La volta scorsa vedemmo che ci sono alcuni strumenti che ci aiutano a lavorare con i dati testuali, come le regular expression e spacy.

Le prime sono molto personalizzabili ma c'è bisogno che i dati abbiamo un certo schema. Mentre con spacy c'è il vincolo che il NER riconosce solo i tipi e le intentità per le quali è stato addestrato.

**Come fare per soddisfare entrambe le esigenze ?**

Possiamo usare un algoritmo chiamato **Conditional Random Field (CRF)**.

Vedremo prima come preparare i dati per il training e successivamente cosa possiamo farci fino ad arrivare alla costruzione di un Bot con intelligenza artificiale.

Il contenuto testuale è stato inserito in uno spreadsheet per una consultazione più agevole.

In [1]:
import numpy as np
import pandas as pd
import re
import os
import random
import pprint
from collections import defaultdict

In [2]:
def remove_nan(df:pd.DataFrame) -> dict:
    
    """
    Rimuove i valori nulli da una lista
    """

    lookup_dict = df.to_dict('list')

    for k, v in lookup_dict.items():

        while np.nan in lookup_dict[k]:
            lookup_dict[k].remove(np.nan)

    return lookup_dict

In [143]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## Parte 1

### Costruzione delle tabelle di lookup

Servono per generare nuovi dati ricombinando elementi già noti. 

In [3]:
df_entities = pd.read_excel('dataset.xlsx',sheet_name='entities_slots')

lookups = remove_nan(df_entities)

In [4]:
pprint.pprint(lookups)

{'CATEGORIA': ['ALLEVAMENTO', 'FRUTTA', 'FORMAGGI'],
 'PROVINCIA': ['NAPOLI', 'CASERTA', 'SALERNO', 'BENEVENTO', 'AVELLINO'],
 'SEDE': ['EBOLI',
          'SOMMA VESUVIANA',
          'NAPOLI',
          'CASERTA',
          'SALERNO',
          'BENEVENTO',
          'AVELLINO',
          'VICO EQUENSE',
          'ASCEA',
          'ACERNO',
          'ALVIGNANO',
          'PASTORANO',
          'ROCCABASCERANA',
          'TRAMONTI',
          'MONTECORVINO ROVELLA',
          'CAPACCIO',
          'NOCERA INFERIORE',
          'MONTECALVO IRPINO',
          'CIORLANO',
          'MONTELLA',
          'MONTECORVINO PUGLIANO']}


### Estrazione frasi utente

Sono le frasi comunemente usate dagli utenti. Verranno ricombinate con le tabelle di lookup.

In [5]:
df_user = pd.read_excel('dataset.xlsx',sheet_name='user', header=None)
df_user.columns = ['user','sentences']
df_user['user'] = df_user['user'].fillna(method='ffill', axis=0)

sentences = defaultdict(list)

df_grouped = df_user.groupby('user')

for group in df_grouped.groups:
    
    sentences[group] = df_grouped.get_group(group)['sentences'].tolist()

In [6]:
# pprint.pprint(sentences)

### Generazione del file txt contenente le frasi

Estrae le frasi dal file Excel e le inserisce in un file txt.

In [11]:
sentences_file = "sentences_origin.txt"

In [8]:
i = True

for k, v in sentences.items():
    
    for values in v:
        
        if i:
            with open(sentences_file, 'w+') as f:
                f.writelines(f'{values}|{k}\n')
                i = False
        else:
            with open(sentences_file, 'a+') as f:
                f.writelines(f'{values}|{k}\n')

### Estrazione frasi bot

Elenco delle frasi che il bot può usare dopo aver compreso la frase dell'utente.

In [6]:
# df_bot = pd.read_excel('dataset.xlsx',sheet_name='bot')
# df_bot.head()


df_bot = pd.read_excel('dataset.xlsx',sheet_name='bot', header=None)
df_bot.columns = ['bot','sentences']
df_bot['bot'] = df_bot['bot'].fillna(method='ffill', axis=0)

bot_sentences = defaultdict(list)

df_grouped = df_bot.groupby('bot')

for group in df_grouped.groups:
    
    bot_sentences[group] = df_grouped.get_group(group)['sentences'].tolist()


In [7]:
bot_sentences

defaultdict(list,
            {'bot_ReplyCity ': ['Nella città di [SEDE] ci sono %d fattorie didattiche'],
             'bot_ReplyProv': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche'],
             'bot_ReplyProvWithParams': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste']})

### Estrazione frasi dialogo

Sono la concatenazione di domande e risposte tra utente e bot.

In [8]:
df_dialogs = pd.read_excel('dataset.xlsx',sheet_name='dialogs')
df_dialogs.head()

,Dialog_1,Dialog_2,Dialog_3
0,user_AskProv,user_AskCity,user_AskProvWithParams
1,bot_ReplyProv,bot_ReplyCity,bot_ReplyProvWithParams


In [9]:
dialogs = remove_nan(df_dialogs) 
dialogs

{'Dialog_1': ['user_AskProv', 'bot_ReplyProv'],
 'Dialog_2': ['user_AskCity', 'bot_ReplyCity '],
 'Dialog_3': ['user_AskProvWithParams', 'bot_ReplyProvWithParams']}

### Generazione di nuove frasi ricombinandole con le lookup tables

Si estraggono le frasi e le categorie di appartenenza per archiviarle in due liste separate

In [12]:
sentences, categories = [], []

with open(sentences_file, encoding='utf-8') as f:
    dataset = f.read()
    dataset = dataset.split("\n")

for data in dataset:
    sentence = data.split("|")
    
    if len(sentence) > 1:
        
        # TODO: Lasciare upper ?
        
        sentences.append(sentence[0].upper())
        categories.append(sentence[1])
        
assert len(sentences) == len(categories)

In [13]:
sentences[0]

'CI SONO FATTORIE DIDATTICHE AD [SEDE](EBOLI)?'

In [14]:
sentences_file_generated = "sentences_generated.txt"

In [15]:
"""
Numero di frasi che verranno casualmente generate
"""

n_sentences = 1000

In [16]:
slots = list(lookups.keys())

for i in range(n_sentences):
        
    index = random.randint(0, len(sentences)-1)

    sentence = sentences[index]
    category = categories[index]
    
    for key in slots:
        
        """
        Ogni volta che regex individua lo slot nella frase
            sostituisce il valore con uno estratto in modo casuale
        """
        
        regex_str = fr'\[{key}]\((?P<value>[a-z ]+)\)+'
        
        slot_match = re.compile(regex_str)

        repl = f"[{key}]({random.choice(lookups[key])})"

        sentence = slot_match.sub(repl, sentence)
        
        
        """
        Poi riassocia la categoria di partenza
        """
        
    if i == 0:
        
        with open(sentences_file_generated, 'w+') as f:
            f.writelines(f'{sentence}|{category}\n')
        
    else:
    
        with open(sentences_file_generated, 'a+') as f:
            f.writelines(f'{sentence}|{category}\n')

## Parte 2

...

...

In [17]:
def rimuovi_punteggiatura(text):
    
    """
    I dati di training contengono parentesi quadre e tonde, 
        quindi non vanno eliminate in questa fase
    """
    
    text = re.sub(r'[\.,;:!?]' , " ", text)
    text = re.sub(r'\s+' , " ", text)
    
    return text

In [18]:
with open(sentences_file_generated, encoding='utf-8') as f:
    sentences = f.read()
    sentences = sentences.split("\n")

In [19]:
example = random.choice(sentences)
example, _ = example.split("|")
print("Before:",example)

example = re.sub(r'\[(?P<name>[a-zA-Z_]+)\]|\(|\)+', "", example)
print("After:",example)

Before: QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI [PROVINCIA](SALERNO)?
After: QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI SALERNO?


### Inizio preparazione dati per l'addestramento dell'algoritmo CRF

In [23]:
sentence = sentences[10]
print(f"Fase iniziale: {sentence} \n")

sentence, categ = sentence.split("|")
sentence = rimuovi_punteggiatura(sentence)

print(f"Fase finale: {sentence}")

Fase iniziale: CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI [PROVINCIA](NAPOLI) CON COLTIVAZIONE DI [CATEGORIA](FRUTTA)?|user_AskProvWithParams 

Fase finale: CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI [PROVINCIA](NAPOLI) CON COLTIVAZIONE DI [CATEGORIA](FRUTTA) 


In [24]:
# Uso dei gruppi nominati

regex_str = r'\[(?P<name>[a-zA-Z_]+)\]\((?P<value>[a-zA-Z\' ]+)\)+'
slot_match = re.compile(regex_str)

##### Con il metodo split possiamo confrontare se ogni elemento della lista appartiene allo slot, al valore o a nessuno dei due.

In [25]:
splits = slot_match.split(sentence)
splits

['CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI ',
 'PROVINCIA',
 'NAPOLI',
 ' CON COLTIVAZIONE DI ',
 'CATEGORIA',
 'FRUTTA',
 ' ']

In [26]:
matches = slot_match.findall(sentence)

dct = {k:v for k,v in matches}
dct

{'PROVINCIA': 'NAPOLI', 'CATEGORIA': 'FRUTTA'}

##### L'obiettivo finale è quello di trasportare i dati in colonne dove  ad ogni riga corrisponde una parola e ogni parola può appartenere o meno ad uno slot

In [27]:
for split in splits:
   
    if split in list(dct.values()):
        for value in split.split():
            
            index = list(dct.values()).index(split)
            key = list(dct.keys())[index]
            
            print(value, "->", key)
    elif split == dct.keys():
        pass
    else:
        for splt in split.split():
            print(splt, "->", '0')

CI -> 0
SONO -> 0
FATTORIE -> 0
DIDATTICHE -> 0
NELLA -> 0
PROVINCIA -> 0
DI -> 0
PROVINCIA -> 0
NAPOLI -> PROVINCIA
CON -> 0
COLTIVAZIONE -> 0
DI -> 0
CATEGORIA -> 0
FRUTTA -> CATEGORIA


#### Procediamo ad applicare la trasformazione a tutto il dataset

In [28]:
arr_sentences = list()
arr_categories = list()

for n, sentence in enumerate(sentences):

    try:
    
        sentence, categ = sentence.split("|")
        
        sentence = rimuovi_punteggiatura(sentence)

        arr_categories.append(categ)

        splits = slot_match.split(sentence)

#         match = slot_match.search(frase)
        matches = slot_match.findall(sentence)
    
        dct = {k:v for k,v in matches}        

        if matches is not None:
            
            for split in splits:
                if split in list(dct.values()):
                    for value in split.split():
                        
                        index = list(dct.values()).index(split)
                        key = list(dct.keys())[index]
                        
                        arr_sentences.append([n, value, key])
                elif split in list(dct.keys()):
                    pass
                else:
                    for value in split.split():
                        arr_sentences.append([n, value, 'O'])
                        
        else:
            """
            Serve per verificare se in qualche frase non avviene il match
            """
            print(n, frase)
            
    except Exception as err:
        pass


arr_sentences[:10]

[[0, 'CI', 'O'],
 [0, 'SONO', 'O'],
 [0, 'FATTORIE', 'O'],
 [0, 'DIDATTICHE', 'O'],
 [0, 'IN', 'O'],
 [0, 'PROVINCIA', 'O'],
 [0, 'DI', 'O'],
 [0, 'SALERNO', 'PROVINCIA'],
 [1, 'QUANTE', 'O'],
 [1, 'FATTORIE', 'O']]

In [29]:
df = pd.DataFrame(arr_sentences, columns=['n_frase','word','tag'])
df.head(10)

,n_frase,word,tag
0,0,CI,O
1,0,SONO,O
2,0,FATTORIE,O
3,0,DIDATTICHE,O
4,0,IN,O
5,0,PROVINCIA,O
6,0,DI,O
7,0,SALERNO,PROVINCIA
8,1,QUANTE,O
9,1,FATTORIE,O


#### Preparazione della variabile target ( y per renderla più familiare... )

In [30]:
df_target = df[['n_frase','tag']]
df_target.head(10)

,n_frase,tag
0,0,O
1,0,O
2,0,O
3,0,O
4,0,O
5,0,O
6,0,O
7,0,PROVINCIA
8,1,O
9,1,O


In [31]:
y = list()

for k, v in df_target.groupby('n_frase'):
        
    y.append(v['tag'].tolist())

In [32]:
y[:5]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA']]

##### Adesso trasformiamo tutto in una funzione

In [33]:
def prepare_target_crf(df:pd.DataFrame) -> list:
    
    """
    Ultimo step per preparare i target per addestrare l'algoritmo
    
    Parameters:
    -----------
    
    df : pd.DataFrame
    
        il DataFrame deve contenere due colonne, una che indicizza la frase
        e l'altra che indica se il valore è uno slot o meno
    
    
    Returns:
    -----------
    y : list
    
        una lista annidata
    
    """
    
    y = list()

    for k, v in df.groupby('n_frase'):

        y.append(v['tag'].tolist())
    
    return y

In [34]:
y = prepare_target_crf(df_target)
y[:5]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA']]

In [35]:
import spacy

nlp = spacy.load("it_core_news_lg")

In [36]:
def spacy_entities_extractor(txt):
    
    doc = nlp(txt.capitalize())
    token = doc[0]
    
    if token.ent_type_ != '':
        return token.ent_type_ # pos_
    else:
        return 'O'

#### Data Augmentation

...spiegare meglio il motivo...

In [37]:
df_sample = df.head(100).copy()

In [38]:
df_sample['shift-3'] = df_sample.groupby('n_frase')['word'].shift(1).str.slice(-3)
df_sample['shift+3'] = df_sample.groupby('n_frase')['word'].shift(-1).str.slice(0,3)
df_sample['shift-3'].fillna('BOF', inplace=True)
df_sample['shift+3'].fillna('EOF', inplace=True)

df_sample['spacy'] = df_sample['word'].apply(spacy_entities_extractor)

df_sample['bias'] = 1

df_sample.head(10)

,n_frase,word,tag,shift-3,shift+3,spacy,bias
0,0,CI,O,BOF,SON,O,1
1,0,SONO,O,CI,FAT,O,1
2,0,FATTORIE,O,ONO,DID,O,1
3,0,DIDATTICHE,O,RIE,IN,O,1
4,0,IN,O,CHE,PRO,O,1
5,0,PROVINCIA,O,IN,DI,O,1
6,0,DI,O,CIA,SAL,O,1
7,0,SALERNO,PROVINCIA,DI,EOF,LOC,1
8,1,QUANTE,O,BOF,FAT,O,1
9,1,FATTORIE,O,NTE,DID,O,1


In [37]:
# for k, frase in df.groupby(by='n_frase')['word']:
#     print(" ".join(frase))
#     print(frase)

In [39]:
def extend_data(df:pd.DataFrame, spacy:bool=False) -> pd.DataFrame:
    
    """
    Estende i dati attraverso un algortimo personalizzato
    """
    
    df['shift-3'] = df.groupby('n_frase')['word'].shift(1).str.slice(-3)
    df['shift+3'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,3)
    df['shift-10'] = df.groupby('n_frase')['word'].shift(1).str.slice(-10)
    df['shift+10'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,10)

    df['shift-3'].fillna('BOF', inplace=True)
    df['shift+3'].fillna('EOF', inplace=True)
    df['shift-10'].fillna('BOF', inplace=True)
    df['shift+10'].fillna('EOF', inplace=True)
    
    if spacy:
        df['spacy'] = df['word'].apply(spacy_entities_extractor)

    df['bias'] = 1
    
    try:
        df.drop(columns=['tag'], inplace=True)
    except:
        pass

    
    return df

In [40]:
df = extend_data(df, spacy=False)
df.head()

,n_frase,word,shift-3,shift+3,shift-10,shift+10,bias
0,0,CI,BOF,SON,BOF,SONO,1
1,0,SONO,CI,FAT,CI,FATTORIE,1
2,0,FATTORIE,ONO,DID,SONO,DIDATTICHE,1
3,0,DIDATTICHE,RIE,IN,FATTORIE,IN,1
4,0,IN,CHE,PRO,DIDATTICHE,PROVINCIA,1


#### Preparazione dei varibili di training ( X per renderli più familiari... )

In [41]:
X = list()

for k, v in df.groupby('n_frase'):
    
    v.drop(columns='n_frase', inplace=True)

    X.append(v.to_dict('records'))

In [42]:
X[0]

[{'word': 'CI',
  'shift-3': 'BOF',
  'shift+3': 'SON',
  'shift-10': 'BOF',
  'shift+10': 'SONO',
  'bias': 1},
 {'word': 'SONO',
  'shift-3': 'CI',
  'shift+3': 'FAT',
  'shift-10': 'CI',
  'shift+10': 'FATTORIE',
  'bias': 1},
 {'word': 'FATTORIE',
  'shift-3': 'ONO',
  'shift+3': 'DID',
  'shift-10': 'SONO',
  'shift+10': 'DIDATTICHE',
  'bias': 1},
 {'word': 'DIDATTICHE',
  'shift-3': 'RIE',
  'shift+3': 'IN',
  'shift-10': 'FATTORIE',
  'shift+10': 'IN',
  'bias': 1},
 {'word': 'IN',
  'shift-3': 'CHE',
  'shift+3': 'PRO',
  'shift-10': 'DIDATTICHE',
  'shift+10': 'PROVINCIA',
  'bias': 1},
 {'word': 'PROVINCIA',
  'shift-3': 'IN',
  'shift+3': 'DI',
  'shift-10': 'IN',
  'shift+10': 'DI',
  'bias': 1},
 {'word': 'DI',
  'shift-3': 'CIA',
  'shift+3': 'SAL',
  'shift-10': 'PROVINCIA',
  'shift+10': 'SALERNO',
  'bias': 1},
 {'word': 'SALERNO',
  'shift-3': 'DI',
  'shift+3': 'EOF',
  'shift-10': 'DI',
  'shift+10': 'EOF',
  'bias': 1}]

##### Adesso trasformiamo tutto in una funzione

In [43]:
def prepare_data_crf(df:pd.DataFrame) -> list:
    
    """
    Ultimo step per preparare i dati per addestrare l'algoritmo
    
    Parameters:
    -----------
    
    df : pd.DataFrame
    
        il numero di colonne del DataFrame dipende da come è stata impostata
        la Data Augmentation... l'importante è che il df contenga
        la colonna 'n_frase' usata come indice
    
    
    Returns:
    -----------
    y : list
    
        una lista annidata di dictionary
    
    """

    
    X = list()

    for k, v in df.groupby('n_frase'):

        v.drop(columns='n_frase', inplace=True)

        X.append(v.to_dict('records'))
    
    return X

In [44]:
X = prepare_data_crf(df)

#### L'algoritmo di Conditional Random Field

In [45]:
import sklearn_crfsuite

In [46]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_states=False,     # Default
    all_possible_transitions=False # Default
)

In [181]:
crf.fit(X, y)

/home/ubuntupc/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_states=False,
    all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
    calibration_candidates=None, calibration_eta=None,
    calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<IPython.core.display.Javascript object>

#### Un esempio su una frase del trainset

In [48]:
sentences[5].split("|")[0]

'CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI [PROVINCIA](NAPOLI) CHE PRODUCONO [CATEGORIA](FORMAGGI)?'

In [49]:
for n, word in enumerate(X[5]):
    
    print(f"{n} --> {word['word']}")

0 --> CI
1 --> SONO
2 --> FATTORIE
3 --> DIDATTICHE
4 --> NELLA
5 --> PROVINCIA
6 --> DI
7 --> NAPOLI
8 --> CHE
9 --> PRODUCONO
10 --> FORMAGGI


In [50]:
y_pred = crf.predict_single(X[5])

for n, pred in enumerate(y_pred):
    
    print(f"{n} --> {pred}")

0 --> O
1 --> O
2 --> O
3 --> O
4 --> O
5 --> O
6 --> O
7 --> PROVINCIA
8 --> O
9 --> O
10 --> CATEGORIA


#### Test su una nuova frase

In [51]:
nuova_frase = 'CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA'

In [52]:
def prepare_sentence(sentence:str) -> [pd.DataFrame, list]:
    
    """
    Prepare la frase per il predict
    
    Parameters:
    -----------
    
    sentence : str
    
        è la frase che sarà elaborata
    
    
    Returns:
    -----------
    df : DataFrame
    
    X_arr[0] : array
        
        dati in formato utile a CRF per il predict
    
    """

    sentence = rimuovi_punteggiatura(sentence)
    
    X_arr = list()
    
    df = pd.DataFrame(data = [i for i in sentence.split()], columns=['word'])
    df['n_frase'] = 1
    
    df = extend_data(df)
    
    
    for k, v in df.groupby('n_frase'):

        v.drop(columns='n_frase', inplace=True)

        X_arr.append(v.to_dict('records'))
    
    
    return df, X_arr[0]

In [53]:
new_df = pd.DataFrame()

new_df = pd.DataFrame({'n_frase':1,
                       'word':[i for i in nuova_frase.split()]})

In [54]:
df, X_arr = prepare_sentence(nuova_frase)

crf.predict_single(X_arr)

['O', 'O', 'O', 'O', 'O', 'SEDE']

##### L'algoritmo ha classificato bene... costruiamo adesso lo "Slots extractor"

In [55]:
def extend_sentence(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> pd.DataFrame:
    
   
    """
    Estrae slots e lo aggiunge al DataFrame come colonna
    
    Parameters:
    -----------
    
    sentence : str
    
        è la stringa che contiene la frase
        
    model : sklearn_crfsuite.estimator.CRF
        
        è il modello addestrato di ConditionalRandomField
    
    
    Returns:
    -----------
    df : DataFrame
    
        al DataFrame di partenza viene aggiunta una colonna 
        con l'indicazione del tipo di slot individuato
       
    """
    
    
    df, X_arr = prepare_sentence(sentence)
    
    df['slots'] = model.predict_single(X_arr)
    
    return df

In [56]:
extend_sentence(nuova_frase, crf)

,word,n_frase,shift-3,shift+3,shift-10,shift+10,bias,slots
0,CI,1,BOF,SON,BOF,SONO,1,O
1,SONO,1,CI,FAT,CI,FATTORIE,1,O
2,FATTORIE,1,ONO,DID,SONO,DIDATTICHE,1,O
3,DIDATTICHE,1,RIE,AD,FATTORIE,AD,1,O
4,AD,1,CHE,AFR,DIDATTICHE,AFRAGOLA,1,O
5,AFRAGOLA,1,AD,EOF,AD,EOF,1,SEDE


#### L'uso di Duckling per il parsing delle date e non solo...

##### N.B.: spiegare duckling partendo dall'utilizzo tramite immagine Docker ma valutare la spiegazione del codice in base al tempo residuo per l'esposizione

In [70]:
import requests
import json
import datetime
import dateparser

def extract_datetime(text:str, url = "http://0.0.0.0:8000/parse"):
    
    data = {"locale":"it_IT",
        "text":text}
    
    resp = None
    
    datetimes = list()
    
    try:
    
        response = requests.post(url, data=data)
        
        try:
        
            if response.status_code == 200:

                for dt in response.json():
                    
                    if dt['dim'] == "time":

                        dtime = dt['value']['value']
                        dtime = dateparser.parse(dtime)

                        datetimes.append(dtime)

            resp = dict()
            
            if len(datetimes) > 1:
                resp['datetime'] = list([min(datetimes), max(datetimes)])
            else:
                resp['datetime'] = list(datetimes)
            
        except:
            pass
        
    except:
        pass
    
    
    return resp

In [71]:
def slots_extractor(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> defaultdict:
    
    
    """
    Restituisce un dictionary degli slots individuati
    
    Parameters:
    -----------
    
    sentence : str
    
        è la stringa che contiene la frase
        
    model : sklearn_crfsuite.estimator.CRF
        
        è il modello addestrato di ConditionalRandomField
    
    
    Returns:
    -----------
    dd : dictionary
           
    """
    
    df = extend_sentence(sentence, model)
    
    dd = defaultdict(list)

    for k, v in df.query("slots != 'O'").groupby('slots'):
        dd[k] = " ".join(v['word'])
        
    
    
    """
    Estrazione date ed orari tramite duckling    
    """
    
    date_time = extract_datetime(sentence)

    if date_time is not None and len(date_time) > 0:
        dd['DATETIMES'] = date_time['datetime']
#     else:
#         sentence_dict['datetimes'] = False


    return dd

In [72]:
slots_extractor('CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 ?', crf)

defaultdict(list,
            {'CATEGORIA': '8',
             'DATETIMES': [datetime.datetime(2021, 11, 21, 8, 0, tzinfo=<StaticTzInfo 'UTC\-08:00'>)]})

## Parte 3

### La costruzione dell'algoritmo del la classificazione degli intents

L'addestramento deve avvenire con le frasi "ripulite" dai dati per addestrare il ConditionalRandomField... ovvero useremo solo gli slots per diminuire lo spazio dimensionale

In [182]:
df = pd.read_csv(sentences_file_generated, sep="|", header=None)

df.columns = ["sentences", "intents"]
df.head()

,sentences,intents
0,CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [P...,user_AskProv
1,QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCI...,user_AskProvWithParams
2,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv
3,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProvWithParams
4,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv


<IPython.core.display.Javascript object>

In [183]:
def conserva_solo_slot_name(text: str) -> str:

    """
    Per facilitare la riduzione dello spazio dimensionale
        vengono eliminati i valori degli slots
        mentre vengono conservati i loro nomi


    Parameters:
    -----------

    text : str

        è la stringa che contiene la frase


    Returns:
    -----------
    text : str

    """

    text = rimuovi_punteggiatura(text)

    pattern = r"(\([A-Za-z0-9 ]+\)|\[|\])"

    text = re.sub(pattern, "", text)

    return text

<IPython.core.display.Javascript object>

In [184]:
df["sentences"][5]

'CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI [PROVINCIA](NAPOLI) CHE PRODUCONO [CATEGORIA](FORMAGGI)?'

<IPython.core.display.Javascript object>

In [185]:
conserva_solo_slot_name(df["sentences"][100])

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PROVINCIA CON PRODUZIONE DI CATEGORIA '

<IPython.core.display.Javascript object>

In [186]:
df["sentences"] = df["sentences"].apply(conserva_solo_slot_name)

df.head()

,sentences,intents
0,CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PR...,user_AskProv
1,QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCI...,user_AskProvWithParams
2,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv
3,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProvWithParams
4,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv


<IPython.core.display.Javascript object>

#### Funzioni per addestrare l'algoritmo di classificazione

...le due funzioni che seguono si spiegano meglio con un esempio...

In [187]:
def replace_slot_values(sentence_dict: dict) -> dict:

    """
    Integrazione dictionary - parte 1 di 2


    Per operare una riduzione delle variabili
    sostituisce il valore con il relativo slot
    """

    sentence_dict["replaced_sentence"] = sentence_dict["sentence"]

    for k, v in sentence_dict["slots"].items():

        if k != "DATETIMES":

            sentence_dict["replaced_sentence"] = re.sub(
                v, k, sentence_dict["replaced_sentence"]
            )

    return sentence_dict

<IPython.core.display.Javascript object>

In [188]:
def add_slots(sentence: str, model: sklearn_crfsuite.estimator.CRF) -> dict:

    """
    Integrazione dictionary - parte 2 di 2
    """

    sentence_dict = {}

    sentence = rimuovi_punteggiatura(sentence)

    slots_dict = slots_extractor(sentence, model)

    sentence_dict["sentence"] = sentence
    sentence_dict["slots"] = slots_dict

    sentence_dict = replace_slot_values(sentence_dict)

    #     date_time = extract_datetime(sentence)

    #     if date_time is not None and len(date_time) > 0:
    #         sentence_dict['datetimes'] = date_time['datetime']
    #     else:
    #         sentence_dict['datetimes'] = False

    return sentence_dict

<IPython.core.display.Javascript object>

In [189]:
nuova_frase

'CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA'

<IPython.core.display.Javascript object>

In [81]:
add_slots('CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI SERA ?', crf)

{'sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI SERA ',
 'slots': defaultdict(list,
             {'CATEGORIA': 'SERA',
              'DATETIMES': [datetime.datetime(2021, 11, 21, 20, 0, tzinfo=<StaticTzInfo 'UTC\-08:00'>)]}),
 'replaced_sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI CATEGORIA '}

#### Inizio preparazione dati per training modello di classificazione

In [190]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

<IPython.core.display.Javascript object>

In [191]:
cv.fit(df.sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

<IPython.core.display.Javascript object>

In [192]:
print(f"Il vocabolario contiene {len(cv.vocabulary_)} parole")

cv.vocabulary_

Il vocabolario contiene 17 parole


{'ci': 3,
 'sono': 16,
 'fattorie': 8,
 'didattiche': 7,
 'in': 9,
 'provincia': 13,
 'di': 6,
 'quante': 14,
 'con': 5,
 'categoria': 1,
 'nella': 10,
 'coltivazione': 4,
 'che': 2,
 'producono': 11,
 'produzione': 12,
 'sede': 15,
 'ad': 0}

<IPython.core.display.Javascript object>

#### Generazione di frasi fake

Aggiungere una classe in più è necessario in quanto l'algoritmo assegna sempre una categoria a qualsiasi testo. Inserendo la classe "fake" l'algoritmo potrà usarla come classe residuale ogni qual volta non riesce ad assegnarla alle 3 classi di nostro interesse. Così facendo il sistema risponderà di non aver capito invitando l'utente a riformulare meglio la frase.

Più nello specifico, l'algoritmo assegna una probabilità ad ogni classe. Viene stabilito un valore soglia. Se la probabilità associata alla classe è superiore alla soglia si accetta la classe, altrimenti si assegna comunque la classe fake.

In [193]:
fake_list = list()

for i in range(30):
    fake_list.append(" ".join(random.choices(list(cv.vocabulary_.keys()), k=10)))

<IPython.core.display.Javascript object>

In [194]:
fake_list[:10]

['nella ad in fattorie produzione sono didattiche sono producono didattiche',
 'con categoria sono in sono ci produzione sede produzione con',
 'che provincia produzione ci provincia che ad sono produzione in',
 'categoria in che provincia ad sede sede categoria con di',
 'provincia categoria provincia categoria quante quante sede che con sede',
 'coltivazione produzione producono di fattorie di di fattorie fattorie provincia',
 'producono categoria didattiche in coltivazione che sono fattorie sono ci',
 'ci che ci di producono provincia che che coltivazione didattiche',
 'producono quante di provincia che in con fattorie categoria sede',
 'producono sede sede sede provincia in sono producono provincia quante']

<IPython.core.display.Javascript object>

In [195]:
df_fake = pd.DataFrame({"sentences": fake_list, "intents": "fake"})
df_fake.head()

,sentences,intents
0,nella ad in fattorie produzione sono didattich...,fake
1,con categoria sono in sono ci produzione sede ...,fake
2,che provincia produzione ci provincia che ad s...,fake
3,categoria in che provincia ad sede sede catego...,fake
4,provincia categoria provincia categoria quante...,fake


<IPython.core.display.Javascript object>

In [196]:
df = pd.concat([df, df_fake], axis=0, ignore_index=True)
df.tail()

,sentences,intents
1025,coltivazione categoria didattiche con ci con q...,fake
1026,didattiche ad coltivazione fattorie fattorie n...,fake
1027,con quante producono coltivazione didattiche s...,fake
1028,sono sono che ci sede in provincia nella categ...,fake
1029,sede producono di producono sono fattorie prod...,fake


<IPython.core.display.Javascript object>

In [197]:
import sklearn

<IPython.core.display.Javascript object>

#### Preparazione del target per il training

In [198]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

<IPython.core.display.Javascript object>

In [199]:
le.fit(df.intents.unique())

for n, cls in enumerate(le.classes_):
    print(f"{cls} --> {n}")

fake --> 0
user_AskCity --> 1
user_AskProv --> 2
user_AskProvWithParams --> 3


<IPython.core.display.Javascript object>

In [200]:
labels_categories = le.transform(df.intents.values)

<IPython.core.display.Javascript object>

In [201]:
le.inverse_transform([1])[0]

'user_AskCity'

<IPython.core.display.Javascript object>

#### Trasformazione e preprocessing della frase per il training del modello

In [202]:
print(df.sentences[0])

np.max(cv.transform(df.sentences[0].split()).toarray(), axis=0)

CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PROVINCIA 


array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1])

<IPython.core.display.Javascript object>

In [139]:
print(f"""
Il dataset si compone di {cv.transform(df['sentences'].values).toarray().shape[0]} frasi
con n.{cv.transform(df['sentences'].values).toarray().shape[1]} colonne 
(infatti il vocabolario contiene 17 parole)
""")


Il dataset si compone di 1030 frasi
con n.17 colonne 
(infatti il vocabolario contiene 17 parole)



In [203]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(fit_intercept=False, loss="log", random_state=200)

<IPython.core.display.Javascript object>

In [204]:
classifier.fit(X=cv.transform(df["sentences"].values).toarray(), y=labels_categories)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=False,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=200, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

<IPython.core.display.Javascript object>

#### Esempio su una frase contenuta nel training set

In [205]:
df["intents"].values[0]

'user_AskProv'

<IPython.core.display.Javascript object>

In [206]:
df["sentences"].values[0]

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PROVINCIA '

<IPython.core.display.Javascript object>

In [207]:
y_pred = classifier.predict(
    [np.max(cv.transform(df["sentences"].values[0].split()).toarray(), axis=0)]
)

<IPython.core.display.Javascript object>

In [208]:
le.inverse_transform(y_pred)[0]

'user_AskProv'

<IPython.core.display.Javascript object>

In [209]:
from sklearn.metrics import confusion_matrix

<IPython.core.display.Javascript object>

In [210]:
predicted_categories = [
    classifier.predict([np.max(cv.transform(sentence.split()).toarray(), axis=0)])[0]
    for sentence in df.sentences.values
]

<IPython.core.display.Javascript object>

In [211]:
cm = confusion_matrix(labels_categories, predicted_categories)
cm

array([[ 29,   0,   0,   1],
       [  0, 224,   0,   0],
       [  0,   0, 316,   0],
       [  0,   0,   0, 460]])

<IPython.core.display.Javascript object>

In [212]:
print(f"Accuracy: {sum(cm.diagonal()) / cm.sum() * 100}%")

Accuracy: 99.90291262135922%


<IPython.core.display.Javascript object>

Se durante la spiegazione live il modello, molto semplice nella sua costruzione, ottiene dei valori non soddisfacenti potrebbe essere interessante vedere come migliorarlo modificando i dati di input (da valutare in base al tempo residuo) altrimenti utilizzarlo comunque facendo leva sul valore soglia descritto in seguito.

#### Test classificazione su nuova frase

In [213]:
processed_sentence = add_slots(nuova_frase, crf)["replaced_sentence"]
processed_sentence

'CI SONO FATTORIE DIDATTICHE AD SEDE'

<IPython.core.display.Javascript object>

In [215]:
y_pred = classifier.predict(
    [np.max(cv.transform(processed_sentence.split()).toarray(), axis=0)]
)[0]

le.inverse_transform([y_pred])[0]

'user_AskCity'

<IPython.core.display.Javascript object>

##### Trasformiamo tutto in una funzione

In [216]:
def get_intents_and_slots(
    sentence: str,
    model: sklearn_crfsuite.estimator.CRF,
    cv: sklearn.feature_extraction.text.CountVectorizer,
    le: sklearn.preprocessing._label.LabelEncoder,
    threasold=0.25,
) -> dict:

    """
    Estrae gli intents e gli slots dalla frase
    """

    sentence_dict = add_slots(sentence, crf)

    sentence = sentence_dict["replaced_sentence"]

    arr = cv.transform(add_slots(sentence, crf)["replaced_sentence"].split()).toarray()

    arr = np.max(arr, axis=0)

    probs = model.predict_proba([arr])[0]

    df = pd.DataFrame({"classes": le.classes_, "probs": probs})
    df.sort_values("probs", ascending=False, inplace=True)
    classes_with_prob = df[df["probs"] > threasold].to_dict("records")
    sentence_dict["intents"] = classes_with_prob

    max_intent = le.classes_[np.argmax(probs)]
    sentence_dict["max_intent"] = max_intent

    return sentence_dict

<IPython.core.display.Javascript object>

In [217]:
data = get_intents_and_slots(nuova_frase, classifier, cv, le)

<IPython.core.display.Javascript object>

In [218]:
pprint.pprint(data)

{'intents': [{'classes': 'user_AskCity', 'probs': 0.9977912681843376}],
 'max_intent': 'user_AskCity',
 'replaced_sentence': 'CI SONO FATTORIE DIDATTICHE AD SEDE',
 'sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA',
 'slots': defaultdict(<class 'list'>, {'SEDE': 'AFRAGOLA'})}


<IPython.core.display.Javascript object>

## Parte 4

### Combinare le info estratte dalla frase per filtrare un semplice dataset OpenData

Comprensione del dialogo e della costruzione della risposta

In [219]:
dialogs = {v[0]: v[1] for v in list(dialogs.values())}

pprint.pprint(dialogs)

{'user_AskCity': 'bot_ReplyCity ',
 'user_AskProv': 'bot_ReplyProv',
 'user_AskProvWithParams': 'bot_ReplyProvWithParams'}


<IPython.core.display.Javascript object>

In [220]:
bot_sentences

defaultdict(list,
            {'bot_ReplyCity ': ['Nella città di [SEDE] ci sono %d fattorie didattiche'],
             'bot_ReplyProv': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche'],
             'bot_ReplyProvWithParams': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste']})

<IPython.core.display.Javascript object>

In [221]:
"""
Estrazione intent
"""

if data["intents"][0]["probs"] > 0.75:

    intent = data["max_intent"]

else:

    intent = "fake"

intent

'user_AskCity'

<IPython.core.display.Javascript object>

In [100]:
# data['intents'][0]['probs'] > 0.75

La costruzione del mini-dialogo verrà fatta con una concatenazione di dictionary anche se il modo migliore è l'uso di una RNN, ma non verrà impiegata per i motivi sopra indicati.

In [222]:
"""
Estrazione risposta
"""

try:

    reply = dialogs[intent]

except:

    reply = "fake"

reply

'bot_ReplyCity '

<IPython.core.display.Javascript object>

In [234]:
"""
Estrazione frase con parametri
"""

if reply != "fake":

    reply_str = random.choice(bot_sentences[reply])

reply_str

print(reply_str, "\n")

"""
Estrazione dei dati per la sostituzione dei parametri
"""

query_list = list()

for k, v in data["slots"].items():
    query_list.append(f"{k} == '{v}'")

    k = list(data["slots"].keys())[0]
    k = f"[{k}]"

    v = list(data["slots"].values())[0]

    reply_str = re.sub(f"[{k}+]", v, reply_str)

query = " and ".join(query_list)

print(query)

Nella città di [SEDE] ci sono %d fattorie didattiche 

SEDE == 'AFRAGOLA'


<IPython.core.display.Javascript object>

### Il file "db_esempio" è stato ottenuto rielaborando un dataset scaricato sul portale degli OpenData disponibile al seguente [https://dati.regione.campania.it/catalogo/resources/Fattorie-didattiche.csv](https://dati.regione.campania.it/catalogo/resources/Fattorie-didattiche.csv)

In [225]:
df_query = pd.read_csv("db_esempio.csv")
df_query.head()

,NOME,SEDE,PROVINCIA,CATEGORIA
0,Azienda Sperimentale Regionale Improsta,EBOLI,SALERNO,ALLEVAMENTO
1,"Museo della Civiltà Contadina ""Michele Russo""",SOMMA VESUVIANA,NAPOLI,ALLEVAMENTO
2,Masseria Panico di Eredi Beneduce Ettore,SOMMA VESUVIANA,NAPOLI,ALLEVAMENTO
3,Cooperativa Agrituristica La Ginestra,VICO EQUENSE,NAPOLI,ALLEVAMENTO
4,Agriturismo Costiera Amalfitana,TRAMONTI,SALERNO,ALLEVAMENTO


<IPython.core.display.Javascript object>

In [226]:
n = df_query.query(query)["NOME"].drop_duplicates().count()

print(reply_str % (n))

Nella città di AFRAGOLA ci sono 0 fattorie didattiche


<IPython.core.display.Javascript object>

In [106]:
# if data['max_intent'] == 'user_AskProvWithParams':
    
#     n = df_query.query(query)['NOME'].drop_duplicates().count()
    
#     print(reply_str % (n))
    
# elif data['max_intent'] == 'user_AskProv':
    
#     n = df_query.query(query)['NOME'].drop_duplicates().count()
    
#     print(reply_str % (n))

# elif data['max_intent'] == 'user_AskCity':
    
#     n = df_query.query(query)['NOME'].drop_duplicates().count()
    
#     print(reply_str % (n))

# else:
#     print("Non ho capito")

##### Adesso raggruppiamo tutto in una funzione

In [235]:
def bot_reply(
    sentence: str,
    model: sklearn_crfsuite.estimator.CRF,
    cv: sklearn.feature_extraction.text.CountVectorizer,
    le: sklearn.preprocessing._label.LabelEncoder,
    dialogs: dict,
    bot_sentences: dict,
    db: str = "db_esempio.csv",
    threasold=0.25,
) -> dict:

    """
    Genera la risposta
    """

    sentence = sentence.upper()

    data = get_intents_and_slots(sentence, model, cv, le, threasold)

    #     intent = data['max_intent']

    pprint.pprint(data)

    print("---------------------------------------------")

    if (data["intents"][0]["probs"] > 0.75) and data["intents"][0]["classes"] != "fake":

        intent = data["max_intent"]

        reply = dialogs[intent]

        reply_str = random.choice(bot_sentences[reply])

        query_list = list()

        for k, v in data["slots"].items():

            query_list.append(f"{k} == '{v}'")

            reply_str = re.sub(fr"\[{k}\]", v, reply_str)

        query = " and ".join(query_list)

        df_query = pd.read_csv(db)

        n = df_query.query(query)["NOME"].drop_duplicates().count()

        return reply_str % (n)

    else:

        intent = "Non ho capito, riformula meglio la tua domanda"

        return intent

<IPython.core.display.Javascript object>

In [236]:
# nuova_frase = "Quante fattorie didattiche ci sono in provincia di Caserta" # con allevamento ?"
nuova_frase = "Quante fattorie didattiche ci sono a Caserta"  # con allevamento ?"

bot_reply(nuova_frase, classifier, cv, le, dialogs, bot_sentences)

{'intents': [{'classes': 'user_AskCity', 'probs': 0.9981957885521887}],
 'max_intent': 'user_AskCity',
 'replaced_sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO A SEDE',
 'sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO A CASERTA',
 'slots': defaultdict(<class 'list'>, {'SEDE': 'CASERTA'})}
---------------------------------------------


'Nella città di CASERTA ci sono 1 fattorie didattiche'

<IPython.core.display.Javascript object>

In [117]:
# Quante fattorie didattiche ci sono in provincia di Caserta con allevamento ?

In [127]:
while True:
    
    try:
    
        nuova_frase = input("Chiedi qualcosa:")

        print(bot_reply(nuova_frase, classifier, cv, le, dialogs, bot_sentences))
        
    except KeyboardInterrupt:
        break